In [ ]:
DATA_PATH = r'/home/parking/ml/data/data/kaggle/predict-student-performance-from-game-play/'

TRAIN_PATH = DATA_PATH + 'train.csv'
TRAIN_LABEL_PATH = DATA_PATH + 'train_label.csv'
TEST_PATH = DATA_PATH + 'test.csv'

import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'